In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Concatenate, Conv2DTranspose, MaxPool2D, Flatten, Conv2D, Reshape, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import Callback
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, UpSampling2D, concatenate, Activation
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Conv2DTranspose, Input
from keras.saving import register_keras_serializable

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Dropout, Concatenate, LeakyReLU, Add
from tensorflow.keras.initializers import he_normal

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm 
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
plt.style.use("dark_background")

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler

import properscoring as ps

# scaler = MaxAbsScaler()
scaler = MinMaxScaler(feature_range=(0,1))

# tf.random.set_seed(42)  # to replicate results

2024-08-19 11:10:22.516006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 11:10:22.516048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 11:10:22.517078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-19 11:10:22.523116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-19 11:10:23.612075: W tensorflow/compiler/tf2

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


In [3]:
# Get the list of available physical GPUs
gpus = tf.config.list_physical_devices('GPU')

# Set memory growth for each GPU
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Create MirroredStrategy
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2024-08-19 11:10:27.875580: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79086 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2024-08-19 11:10:27.877155: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 79086 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:44:00.0, compute capability: 8.0
2024-08-19 11:10:27.878572: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 79086 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:84:00.0, compute capability: 8.0
2024-08-19 11:10:27.880046: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 79086 MB memory:  -> device: 3, name: NVIDIA A100-SXM4-80GB, pci bu

In [4]:
diri = '/work/bb0983/athul_satheesh/e_obs_precip/'
diro = '/work/bb0983/athul_satheesh/downscaled_data/europe/figures/'

coarse_raw = 'rr_ens_mean_1.0deg_reg_v29.0e.nc'
fine_raw = 'rr_ens_mean_0.1deg_reg_v29.0e.nc'

lati = 43#40
latf = 59#60

loni = -6#-10
lonf = 15#30

strt = '1950-01-01'
last = '2023-12-31'

coarse_data = xr.open_dataset(diri+coarse_raw).rr.transpose('time','lat','lon').sel(time=slice(strt, last), 
                                                                                    lat=slice(lati, latf), 
                                                                                    lon=slice(loni, lonf)
                                                                                   )
fine_data = xr.open_dataset(diri+fine_raw).rr.transpose('time','latitude','longitude').sel(time=slice(strt, last), 
                                                                                           latitude=slice(lati, latf), 
                                                                                           longitude=slice(loni, lonf)
                                                                                          )
fine_data = fine_data.rename({'latitude':'lat', 'longitude':'lon'})

In [5]:
dims_coarse = coarse_data.shape
dims_fine = fine_data.shape

In [6]:
dims_coarse, dims_fine

((27028, 16, 21), (27028, 160, 210))

In [7]:
train_strt = strt
train_last = '2000-12-31'

test_strt = '2001-01-01'
test_last = last

coarse_data_train = coarse_data.sel(time=slice(train_strt, train_last))
coarse_data_test = coarse_data.sel(time=slice(test_strt, test_last))

fine_data_train = fine_data.sel(time=slice(train_strt, train_last))
fine_data_test = fine_data.sel(time=slice(test_strt, test_last))

In [8]:
coarse_nan_mask = np.isnan(coarse_data_test)
fine_nan_mask = np.isnan(fine_data_test)

fill_val = -1e-10

In [9]:
coarse_data_train = coarse_data_train.fillna(fill_val)
coarse_data_test = coarse_data_test.fillna(fill_val)

fine_data_train = fine_data_train.fillna(fill_val)
fine_data_test = fine_data_test.fillna(fill_val)

In [10]:
# def ssim_loss(fake_images, real_images):
#     """
#     Computes the SSIM loss between fake and real images.
    
#     Parameters:
#         fake_images (tf.Tensor): Generated images.
#         real_images (tf.Tensor): Real images.
    
#     Returns:
#         tf.Tensor: SSIM loss.
#     """
#     ssim_index = tf.image.ssim(fake_images, real_images, max_val=1.0)
#     return 1 - tf.reduce_mean(ssim_index)

In [11]:
def crps_loss(predictions, observed):
    """
    Computes the CRPS loss for an ensemble of predictions against observed data.
    
    Parameters:
        predictions (tf.Tensor): Tensor of shape (batch_size, ensemble_size, height, width, channels)
                                 representing the ensemble of predictions.
        observed (tf.Tensor): Tensor of shape (batch_size, height, width, channels) representing the observed data.
    
    Returns:
        tf.Tensor: The CRPS loss.
    """
    # Sort the predictions to compute the empirical CDF
    sorted_predictions = tf.sort(predictions, axis=1)
    
    # Reshape observed data to match the ensemble shape for broadcasting
    observed = tf.expand_dims(observed, axis=1)
    
    # Calculate the empirical CDF
    cdf = tf.cumsum(tf.ones_like(sorted_predictions) / tf.cast(tf.shape(sorted_predictions)[1], tf.float32), axis=1)
    
    # CRPS calculation
    crps = tf.reduce_mean((cdf - tf.cast(sorted_predictions >= observed, tf.float32))**2, axis=1)
    
    return tf.reduce_mean(crps)

In [12]:
def generator_loss(fake_output, fake_images, real_images, penalty_weight=10, crps_weight=10):
    """
    Generator loss function using Wasserstein loss with an added penalty term and SSIM loss.
    
    Parameters:
        fake_output (tf.Tensor): Output of the discriminator when given generated images.
        fake_images (tf.Tensor): Generated images.
        real_images (tf.Tensor): Real images.
        penalty_weight (float): Weight of the penalty term.
        ssim_weight (float): Weight of the SSIM loss term.
    
    Returns:
        tf.Tensor: Generator loss.
    """
    # print(fake_images.shape)
    # print(real_images.shape)
    # print(tf.reduce_mean(fake_images, axis=-1).shape)
    # print(tf.reshape( tf.reduce_mean(fake_images, axis=-1), real_images.shape + (1,)).shape)

    
    wasserstein_loss = -tf.reduce_mean(fake_output)
    
    # Penalty term for deviation from real outputs
    # penalty = penalty_weight * tf.reduce_mean(tf.abs(fake_output - real_output))
    penalty = penalty_weight * tf.reduce_mean(tf.abs(tf.reduce_mean(fake_images, axis=-1, keepdims=True) - real_images) )
    
    # CRPS loss
    crps_loss_value = crps_loss(fake_images, real_images)
    
    return wasserstein_loss + penalty + crps_weight * crps_loss_value
    # return wasserstein_loss + ssim_weight * ssim_loss_value

In [13]:
def discriminator_loss(real_output, fake_output):
    """
    Discriminator loss function using Wasserstein loss.
    
    Parameters:
        real_output (tf.Tensor): Output of the discriminator when given real images.
        fake_output (tf.Tensor): Output of the discriminator when given generated images.
    
    Returns:
        tf.Tensor: Discriminator loss.
    """
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

In [14]:
# def discriminator_loss(real_output, fake_output, real_images, fake_images, gradient_penalty_weight=10):
#     """
#     Discriminator loss function using Wasserstein loss and gradient penalty.
    
#     Parameters:
#         real_output (tf.Tensor): Output of the discriminator when given real images.
#         fake_output (tf.Tensor): Output of the discriminator when given generated images.
#         real_images (tf.Tensor): Real images.
#         fake_images (tf.Tensor): Generated images.
#         gradient_penalty_weight (float): Weight of the gradient penalty term.
    
#     Returns:
#         tf.Tensor: Discriminator loss.
#     """
#     wasserstein_loss = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)
    
#     # Gradient penalty
#     batch_size = real_images.shape[0]
#     alpha = tf.random.uniform(shape=[batch_size, 1, 1, 1], minval=0., maxval=1.)
#     interpolated = alpha * real_images + (1 - alpha) * fake_images
    
#     with tf.GradientTape() as tape:
#         tape.watch(interpolated)
#         interpolated_output = discriminator(interpolated)
    
#     gradients = tape.gradient(interpolated_output, [interpolated])[0]
#     gradients_norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))
#     gradient_penalty = gradient_penalty_weight * tf.reduce_mean((gradients_norm - 1.0) ** 2)
    
#     return wasserstein_loss + gradient_penalty

In [15]:
# Weight clipping
def clip_weights(model, clip_value):
    """
    Clips the weights of the model to be within the range [-clip_value, clip_value].
    
    Parameters:
        model (tf.keras.Model): The model whose weights will be clipped.
        clip_value (float): The value to clip the weights to.
    """
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            kernel = layer.kernel
            clipped_kernel = tf.clip_by_value(kernel, -clip_value, clip_value)
            layer.kernel.assign(clipped_kernel)

In [16]:
# Use a custom dropout layer that is active also during prediction and use this in the model instead of the default Dropout.
# Alternatively you can also set "training=True" in the Dropout layer
@register_keras_serializable()
class CustomDropout(Dropout):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(rate, **kwargs)

    def call(self, inputs, training=None):
        return super().call(inputs, training=True)  # Always active


In [17]:
def conv_block(x, filters, kernel_size, name, dilation_rate, strides=(1,1), use_batch_norm=True, use_dropout=True):
    x = Conv2D(filters=filters, kernel_size=kernel_size, dilation_rate=dilation_rate, strides=strides, padding='same', 
               kernel_initializer=he_normal(), name=name+'_conv')(x)
    x = LeakyReLU(alpha=0.2, name=name+'_lrelu')(x)
    if use_batch_norm:
        x = BatchNormalization(name=name+'_bn')(x)
    if use_dropout:
        # x = Dropout(0.25, name=name+'_dropout')(x, training=True)
        x = CustomDropout(0.25, name=name+'_dropout')(x)
    return x

def deconv_block(x, filters, kernel_size, strides, name, dilation_rate, use_batch_norm=True, use_dropout=True):
    x = Conv2DTranspose(filters=filters, kernel_size=kernel_size, dilation_rate=dilation_rate, strides=strides, 
                        padding='same', kernel_initializer=he_normal(), name=name+'_deconv')(x)
    x = LeakyReLU(alpha=0.2, name=name+'_lrelu')(x)
    if use_batch_norm:
        x = BatchNormalization(name=name+'_bn')(x)
    if use_dropout:
        x = CustomDropout(0.25, name=name+'_dropout')(x)
    return x

# def upsampling_block(inputs, filters, kernel_size, upsample_factor, name, dilation_rate, strides, use_batch_norm=True, use_dropout=True):
#     x = UpSampling2D(size=upsample_factor, name=name+'_upsample')(inputs)
#     x = Conv2D(filters, kernel_size, dilation_rate=dilation_rate, padding='same', strides=strides,
#                kernel_initializer=he_normal(), name=name+'_conv')(x)
#     x = LeakyReLU(alpha=0.2, name=name+'_lrelu')(x)
#     if use_batch_norm:
#         x = BatchNormalization(name=name+'_bn')(x)
#     if use_dropout:
#         x = CustomDropout(0.25, name=name+'_dropout')(x)
#     return x

In [18]:
# def build_generator(input_shape):
#     inputs = Input(shape=input_shape, name='generator_input')

#     # Downsampling
#     downsample_3 = conv_block(inputs, 512, (3, 3), 'downsample_3', (1,1))
#     downsample_2 = conv_block(downsample_3, 256, (3, 3), 'downsample_2', (1,1))
#     downsample_1 = conv_block(downsample_2, 128, (3, 3), 'downsample_1', (1,1))
#     downsample_0 = conv_block(downsample_1, 64, (3, 3), 'downsample_0', (1,1))

#     # Bottleneck
#     bottleneck_0 = conv_block(downsample_0, 32, (3, 3), 'bottleneck_0', (1,1))
#     bottleneck_00 = conv_block(bottleneck_0, 32, (3, 3), 'bottleneck_00', (1,1))

#     # Upsampling
#     upsample_0 = upsampling_block(bottleneck_00, 64, (3, 3), (1,1), 'upsample_0', (1,1), (1,1))
#     upsample_1 = upsampling_block(upsample_0, 128, (3, 3), (1,1), 'upsample_1', (1,1), (1,1))
#     upsample_2 = upsampling_block(upsample_1, 256, (3, 3), (2,2), 'upsample_2', (1,1), (1,1))
#     upsample_3 = upsampling_block(upsample_2, 512, (6, 6), (5,5), 'upsample_3', (1,1), (1,1))
    
#     # Final Upsample to required shape
#     outputs = Conv2D(filters=1, kernel_size=(3, 3), kernel_initializer=he_normal(), padding='same', 
#                               strides=(1, 1), activation='relu', name='final_upsample_conv')(upsample_3)
    
#     model = Model(inputs=inputs, outputs=outputs, name='generator')
    
#     return model

In [19]:
# def build_generator(input_shape):
#     inputs = Input(shape=input_shape, name='generator_input')

#     # Convolutions
#     conv_3 = conv_block(inputs, 512, (3, 3), 'conv_3', (1,1))
#     conv_2 = conv_block(conv_3, 256, (3, 3), 'conv_2', (1,1))
#     conv_1 = conv_block(conv_2, 128, (3, 3), 'conv_1', (1,1))
#     conv_0 = conv_block(conv_1, 64, (3, 3), 'conv_0', (1,1))
    
#     # Bottleneck
#     bottleneck_0 = conv_block(conv_0, 32, (3, 3), 'bottleneck_0', (1,1))
#     bottleneck_00 = conv_block(bottleneck_0, 32, (3, 3), 'bottleneck_00', (1,1))
    
#     # Upsampling with skip connections
#     deconv_0 = deconv_block(bottleneck_00, 64, (3, 3), (1,1), 'deconv_0', (1,1))
#     deconv_1 = deconv_block(deconv_0, 128, (3, 3), (1,1), 'deconv_1', (1,1))
#     deconv_2 = deconv_block(deconv_1, 256, (3, 3), (2,2), 'deconv_2', (1,1))
#     deconv_3 = deconv_block(deconv_2, 512, (6, 6), (5,5), 'deconv_3', (1,1))
    
#     # Final Upsample to required shape
#     outputs = Conv2D(filters=1, kernel_size=(3, 3), kernel_initializer=he_normal(), padding='same', 
#                               strides=(1, 1), activation='relu', name='final_conv')(deconv_3)
    
#     model = Model(inputs=inputs, outputs=outputs, name='generator')
    
#     return model

In [20]:
def build_generator(input_shape):
    inputs = Input(shape=input_shape, name='generator_input')

    # Convolutions
    conv_4 = conv_block(inputs, 16, (3, 3), 'conv_4', (1,1))
    conv_3 = conv_block(conv_4, 32, (3, 3), 'conv_3', (1,1))
    conv_2 = conv_block(conv_3, 64, (3, 3), 'conv_2', (1,1))
    conv_1 = conv_block(conv_2, 128, (3, 3), 'conv_1', (1,1))
    conv_0 = conv_block(conv_1, 256, (3, 3), 'conv_0', (1,1))
    conv_00 = conv_block(conv_0, 512, (3, 3), 'conv_00', (1,1))
    
    # Upsampling
    deconv_00 = deconv_block(conv_00, 512, (3, 3), (1,1), 'deconv_00', (1,1))
    deconv_0 = deconv_block(deconv_00, 256, (3, 3), (1,1), 'deconv_0', (1,1))
    deconv_1 = deconv_block(deconv_0, 128, (3, 3), (1,1), 'deconv_1', (1,1))
    deconv_2 = deconv_block(deconv_1, 64, (3, 3), (1,1), 'deconv_2', (1,1))
    # deconv_3 = deconv_block(deconv_2, 32, (3, 3), (2,2), 'deconv_3', (1,1))
    deconv_3 = deconv_block(deconv_2, 32, (3, 3), (1,1), 'deconv_3', (1,1))
    deconv_4 = deconv_block(deconv_3, 16, (6, 6), (5,5), 'deconv_4', (1,1))
    
    # Final Upsample to required shape
    # outputs = Conv2D(filters=50, kernel_size=(3, 3), kernel_initializer=he_normal(), padding='same', 
    #                           strides=(1, 1), activation='relu', name='final_conv')(deconv_4)
    outputs = Conv2DTranspose(filters=50, kernel_size=(3, 3), kernel_initializer=he_normal(), padding='same', 
                              strides=(2, 2), activation='relu', name='final_deconv')(deconv_4)
    
    model = Model(inputs=inputs, outputs=outputs, name='generator')
    
    return model

In [21]:
# def build_discriminator(input_shape):
#     input_layer = Input(shape=input_shape, name='discriminator_input')

#     x = conv_block(input_layer, filters=16, kernel_size=(3, 3), name='conv1', dilation_rate=(1,1), strides=(2,2), use_dropout=False, use_batch_norm=False)
#     x = conv_block(x, filters=32, kernel_size=(3, 3), name='conv2', dilation_rate=(1,1), strides=(1,1), use_dropout=False, use_batch_norm=False)
#     x = conv_block(x, filters=64, kernel_size=(3, 3), name='conv3', dilation_rate=(1,1), strides=(1,1), use_dropout=False, use_batch_norm=False)
#     x = conv_block(x, filters=128, kernel_size=(3, 3), name='conv4', dilation_rate=(1,1), strides=(1,1), use_dropout=False, use_batch_norm=False)
#     x = conv_block(x, filters=256, kernel_size=(3, 3), name='conv5', dilation_rate=(1,1), strides=(1,1), use_dropout=False, use_batch_norm=False)
#     x = conv_block(x, filters=512, kernel_size=(3, 3), name='conv6', dilation_rate=(1,1), strides=(1,1), use_dropout=False, use_batch_norm=False)
        
#     x = MaxPooling2D(pool_size=(2,2))(x)
#     x = Dropout(0.25)(x)
#     x = Flatten()(x)
#     output_layer = Dense(1, activation='linear')(x)
#     # output_layer = Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation='linear', name='final_conv')(x)
    
#     model = Model(inputs=input_layer, outputs=output_layer, name='wGAN_discriminator')
    
#     return model

In [22]:
def build_discriminator(input_shape):
    input_layer = Input(shape=input_shape, name='discriminator_input')

    x = conv_block(input_layer, filters=16, kernel_size=(3, 3), name='conv1', dilation_rate=(1,1), strides=(2,2), use_dropout=False)
    # x = conv_block(x, filters=32, kernel_size=(3, 3), name='conv2', dilation_rate=(1,1), strides=(1,1), use_dropout=False)
    x = conv_block(x, filters=64, kernel_size=(3, 3), name='conv3', dilation_rate=(1,1), strides=(1,1), use_dropout=False)
    # x = conv_block(x, filters=128, kernel_size=(3, 3), name='conv4', dilation_rate=(1,1), strides=(1,1), use_dropout=False)
    x = conv_block(x, filters=256, kernel_size=(3, 3), name='conv5', dilation_rate=(1,1), strides=(1,1), use_dropout=False)
    # x = conv_block(x, filters=512, kernel_size=(3, 3), name='conv6', dilation_rate=(1,1), strides=(1,1), use_dropout=False)
        
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.25)(x)
    # output_layer = Dense(1, activation='linear')(x)
    output_layer = Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation='linear', name='final_conv')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer, name='wGAN_discriminator')
    
    return model

In [23]:
input_shape  = dims_coarse[1:] + (1,)
output_shape = dims_fine[1:] + (1,)

# lr = 0.5e-5
gen_lr = 2e-4
dis_lr = 2e-4
clip_value = 1e-3


with strategy.scope():
    
    # gen_optimizer = RMSprop(learning_rate=lr, momentum=0.5)
    # dis_optimizer = RMSprop(learning_rate=lr, momentum=0.5)
    gen_optimizer = Adam(learning_rate=gen_lr)
    dis_optimizer = Adam(learning_rate=dis_lr)
    
    gen = build_generator(input_shape)
    dis = build_discriminator(output_shape)

    gen.compile(optimizer=gen_optimizer,)
    dis.compile(optimizer=dis_optimizer,)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [24]:
gen.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 generator_input (InputLaye  [(None, 16, 21, 1)]       0         
 r)                                                              
                                                                 
 conv_4_conv (Conv2D)        (None, 16, 21, 16)        160       
                                                                 
 conv_4_lrelu (LeakyReLU)    (None, 16, 21, 16)        0         
                                                                 
 conv_4_bn (BatchNormalizat  (None, 16, 21, 16)        64        
 ion)                                                            
                                                                 
 conv_4_dropout (CustomDrop  (None, 16, 21, 16)        0         
 out)                                                            
                                                         

In [25]:
dis.summary()

Model: "wGAN_discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 discriminator_input (Input  [(None, 160, 210, 1)]     0         
 Layer)                                                          
                                                                 
 conv1_conv (Conv2D)         (None, 80, 105, 16)       160       
                                                                 
 conv1_lrelu (LeakyReLU)     (None, 80, 105, 16)       0         
                                                                 
 conv1_bn (BatchNormalizati  (None, 80, 105, 16)       64        
 on)                                                             
                                                                 
 conv3_conv (Conv2D)         (None, 80, 105, 64)       9280      
                                                                 
 conv3_lrelu (LeakyReLU)     (None, 80, 105, 64)

In [26]:
# Convert data to tensor
def preprocess_data(data):
    data = tf.convert_to_tensor(data.values, dtype=tf.float32)
    return tf.reshape(data, data.shape + (1,))
# Compute and update gradients
def train_step(coarse_data_batch, fine_data_batch, dis, clip_value):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
        fake_data_batch = gen(coarse_data_batch, training=True)
        real_output = dis(fine_data_batch, training=True)
        # fake_output = dis(fake_data_batch[...,-1], training=True)
        fake_output = dis( tf.reshape(tf.reduce_mean(fake_data_batch, axis=-1), fine_data_batch.shape + (1,)), training=True) # change to a single member instead of the mean if required
        gen_loss = generator_loss(fake_output, fake_data_batch, fine_data_batch)
        dis_loss = discriminator_loss(real_output, fake_output)

    gradients_of_gen = gen_tape.gradient(gen_loss, gen.trainable_variables)
    gradients_of_dis = dis_tape.gradient(dis_loss, dis.trainable_variables)
    
    gen_optimizer.apply_gradients(zip(gradients_of_gen, gen.trainable_variables))
    dis_optimizer.apply_gradients(zip(gradients_of_dis, dis.trainable_variables))
    
    # Clip discriminator weights
    clip_weights(dis, clip_value)
    
    return gen_loss, dis_loss

@tf.function
def distributed_train_step(coarse_data_batch, fine_data_batch, dis, clip_value):
    per_replica_gen_losses, per_replica_dis_losses = strategy.run(train_step, args=(coarse_data_batch, fine_data_batch, dis, clip_value))
    mean_gen_loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_gen_losses, axis=None)
    mean_dis_loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_dis_losses, axis=None)
    return mean_gen_loss, mean_dis_loss

def train_gan(gen, dis, coarse_data_train, fine_data_train, clip_value, epochs, batch_size, save_intermediate=True):
    dataset = tf.data.Dataset.from_tensor_slices((preprocess_data(coarse_data_train), preprocess_data(fine_data_train)))
    # dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat(epochs)
    dataset = dataset.shuffle(buffer_size=365*2).batch(batch_size)#.repeat(epochs)
    distributed_dataset = strategy.experimental_distribute_dataset(dataset)

    for epoch in range(epochs):
        total_gen_loss = 0.0
        total_dis_loss = 0.0
        num_batches = 0

        for coarse_data_batch, fine_data_batch in distributed_dataset:
            gen_loss, dis_loss = distributed_train_step(coarse_data_batch, fine_data_batch, dis, clip_value)
            total_gen_loss += gen_loss
            total_dis_loss += dis_loss
            num_batches += 1

        avg_gen_loss = total_gen_loss / num_batches
        avg_dis_loss = total_dis_loss / num_batches
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, Generator Loss: {avg_gen_loss:.5f}, Discriminator Loss: {avg_dis_loss:.5f}")

        # Save models every 100 epochs
        if save_intermediate:
            if (epoch + 1) % 50 == 0:
                gen.save(f'/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_extended_eu/generator_prob_epoch_{epoch+1}_adam_crps.keras')
                dis.save(f'/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_extended_eu/discriminator_prob_epoch_{epoch+1}_adam_crps.keras')

In [27]:
# def preprocess_data(data):
#     data = tf.convert_to_tensor(data.values, dtype=tf.float32)
#     return tf.reshape(data, data.shape + (1,))

# # Compute and update gradients
# def train_discriminator_step(coarse_data_batch, fine_data_batch, dis, clip_value):
#     with tf.GradientTape() as dis_tape:
#         fake_data_batch = gen(coarse_data_batch, training=True)
#         real_output = dis(fine_data_batch, training=True)
#         fake_output = dis(fake_data_batch, training=True)
#         dis_loss = discriminator_loss(real_output, fake_output)

#     gradients_of_dis = dis_tape.gradient(dis_loss, dis.trainable_variables)
#     dis_optimizer.apply_gradients(zip(gradients_of_dis, dis.trainable_variables))
    
#     # Clip discriminator weights
#     clip_weights(dis, clip_value)
    
#     return dis_loss

# def train_generator_step(coarse_data_batch, fine_data_batch, dis):
#     with tf.GradientTape() as gen_tape:
#         fake_data_batch = gen(coarse_data_batch, training=True)
#         fake_output = dis(fake_data_batch, training=True)
#         real_output = dis(fine_data_batch, training=True)
#         gen_loss = generator_loss(fake_output, real_output, fake_data_batch, fine_data_batch)

#     gradients_of_gen = gen_tape.gradient(gen_loss, gen.trainable_variables)
#     gen_optimizer.apply_gradients(zip(gradients_of_gen, gen.trainable_variables))
    
#     return gen_loss

# @tf.function
# def distributed_discriminator_step(coarse_data_batch, fine_data_batch, dis, clip_value):
#     per_replica_dis_losses = strategy.run(train_discriminator_step, args=(coarse_data_batch, fine_data_batch, dis, clip_value))
#     mean_dis_loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_dis_losses, axis=None)
#     return mean_dis_loss

# @tf.function
# def distributed_generator_step(coarse_data_batch, fine_data_batch, dis):
#     per_replica_gen_losses = strategy.run(train_generator_step, args=(coarse_data_batch, fine_data_batch, dis))
#     mean_gen_loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_gen_losses, axis=None)
#     return mean_gen_loss

# def train_gan(gen, dis, coarse_data_train, fine_data_train, clip_value, epochs, batch_size, save_intermediate=True, d_steps=5):
#     dataset = tf.data.Dataset.from_tensor_slices((preprocess_data(coarse_data_train), preprocess_data(fine_data_train)))
#     dataset = dataset.shuffle(buffer_size=365*2).batch(batch_size)
#     distributed_dataset = strategy.experimental_distribute_dataset(dataset)

#     for epoch in range(epochs):
#         total_gen_loss = 0.0
#         total_dis_loss = 0.0
#         num_batches = 0

#         for coarse_data_batch, fine_data_batch in distributed_dataset:
#             for _ in range(d_steps):
#                 dis_loss = distributed_discriminator_step(coarse_data_batch, fine_data_batch, dis, clip_value)
#                 total_dis_loss += dis_loss
#                 num_batches += 1

#             gen_loss = distributed_generator_step(coarse_data_batch, fine_data_batch, dis)
#             total_gen_loss += gen_loss

#         avg_gen_loss = total_gen_loss / num_batches
#         avg_dis_loss = total_dis_loss / (num_batches * d_steps)
        
#         if (epoch + 1) % 10 == 0:
#             print(f"Epoch {epoch + 1}, Generator Loss: {avg_gen_loss:.5f}, Discriminator Loss: {avg_dis_loss:.5f}")

#         # Save models every 100 epochs
#         if save_intermediate and (epoch + 1) % 50 == 0:
#             gen.save(f'/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_extended_eu/generator_prob_epoch_{epoch+1}.keras')
#             dis.save(f'/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_extended_eu/discriminator_prob_epoch_{epoch+1}.keras')

In [28]:
epochs = 850
batch_size = 365

In [ ]:
%%time
train_gan(gen, dis, coarse_data_train, fine_data_train, clip_value, epochs, batch_size, save_intermediate=True)

INFO:tensorflow:Collective all_reduce tensors: 50 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 50 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1


2024-08-19 11:11:43.515685: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inreplica_1/generator/conv_4_dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


In [ ]:
# Unset the random seed to generate ensembles
tf.random.set_seed(None) 

downscaled_data_mem1 = gen.predict( coarse_data_test.values.reshape( coarse_data_test.shape + (1,) ) )
downscaled_data_mem2 = gen.predict( coarse_data_test.values.reshape( coarse_data_test.shape + (1,) ) )
downscaled_data_mem3 = gen.predict( coarse_data_test.values.reshape( coarse_data_test.shape + (1,) ) )
downscaled_data_mem4 = gen.predict( coarse_data_test.values.reshape( coarse_data_test.shape + (1,) ) )
downscaled_data_mem5 = gen.predict( coarse_data_test.values.reshape( coarse_data_test.shape + (1,) ) )
downscaled_data_mem6 = gen.predict( coarse_data_test.values.reshape( coarse_data_test.shape + (1,) ) )

In [ ]:
downscaled_data_mem1.shape, downscaled_data_mem2.shape, downscaled_data_mem3.shape, downscaled_data_mem4.shape, downscaled_data_mem5.shape, downscaled_data_mem6.shape

In [ ]:
# downscaled_data = inverse_normalize_data(downscaled_data, scaler)

In [ ]:
downscaled_data_mem1 = xr.DataArray(name='precipitation', data=downscaled_data_mem1.squeeze(), 
                               dims=fine_data_test.dims, coords=fine_data_test.coords, 
                               attrs=fine_data_test.attrs)
downscaled_data_mem1 = downscaled_data_mem1.where(~fine_nan_mask, np.nan)

downscaled_data_mem2 = xr.DataArray(name='precipitation', data=downscaled_data_mem2.squeeze(), 
                               dims=fine_data_test.dims, coords=fine_data_test.coords, 
                               attrs=fine_data_test.attrs)
downscaled_data_mem2 = downscaled_data_mem2.where(~fine_nan_mask, np.nan)

downscaled_data_mem3 = xr.DataArray(name='precipitation', data=downscaled_data_mem3.squeeze(), 
                               dims=fine_data_test.dims, coords=fine_data_test.coords, 
                               attrs=fine_data_test.attrs)
downscaled_data_mem3 = downscaled_data_mem3.where(~fine_nan_mask, np.nan)

downscaled_data_mem4 = xr.DataArray(name='precipitation', data=downscaled_data_mem4.squeeze(), 
                               dims=fine_data_test.dims, coords=fine_data_test.coords, 
                               attrs=fine_data_test.attrs)
downscaled_data_mem4 = downscaled_data_mem4.where(~fine_nan_mask, np.nan)

downscaled_data_mem5 = xr.DataArray(name='precipitation', data=downscaled_data_mem5.squeeze(), 
                               dims=fine_data_test.dims, coords=fine_data_test.coords, 
                               attrs=fine_data_test.attrs)
downscaled_data_mem5 = downscaled_data_mem5.where(~fine_nan_mask, np.nan)

downscaled_data_mem6 = xr.DataArray(name='precipitation', data=downscaled_data_mem6.squeeze(), 
                               dims=fine_data_test.dims, coords=fine_data_test.coords, 
                               attrs=fine_data_test.attrs)
downscaled_data_mem6 = downscaled_data_mem6.where(~fine_nan_mask, np.nan)

In [ ]:
downscaled_data_list = [downscaled_data_mem1, downscaled_data_mem2, downscaled_data_mem3, 
                   downscaled_data_mem4, downscaled_data_mem5, downscaled_data_mem6
                  ]
downscaled_data = xr.concat(downscaled_data_list, dim='number')
downscaled_data = downscaled_data.assign_coords(number=np.arange( len(downscaled_data_list)) )
downscaled_data = downscaled_data.transpose('time', 'lat', 'lon', 'number')

In [ ]:
downscaled_data

In [ ]:
# downscaled_data = downscaled_data.where(~fine_nan_mask, np.nan)
fine_data_test = fine_data_test.where(~fine_nan_mask, np.nan)
coarse_data_test = coarse_data_test.where(~coarse_nan_mask, np.nan)

In [ ]:
diro = '/work/bb0983/athul_satheesh/downscaled_data/europe/downscaled_data/'
# downscaled_data.to_netcdf(diro + 'e_obs_eu_downscaled_wgan_prob.nc')

In [ ]:
# save model
# gen.save('/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_generator_prob.keras')

In [ ]:
# save model
# dis.save('/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_discriminator_prob.keras')

In [ ]:
# ps.crps_ensemble?

In [ ]:
date1 = '2021-06-15'
date2 = '2021-09-15'

In [ ]:
# print(f"CRPS: {np.nanmean(ps.crps_ensemble(observations=fine_data_test, forecasts=downscaled_data) ):.4f}")
print(f"MAE1: {np.nanmean(ps.crps_ensemble(observations=fine_data_test, forecasts=downscaled_data.sel(number=0)) ):.4f}")
print(f"MAE2: {np.nanmean(ps.crps_ensemble(observations=fine_data_test, forecasts=downscaled_data.sel(number=1)) ):.4f}")
print(f"MAE3: {np.nanmean(ps.crps_ensemble(observations=fine_data_test, forecasts=downscaled_data.sel(number=2)) ):.4f}")
print(f"MAE4: {np.nanmean(ps.crps_ensemble(observations=fine_data_test, forecasts=downscaled_data.sel(number=3)) ):.4f}")
print(f"MAE5: {np.nanmean(ps.crps_ensemble(observations=fine_data_test, forecasts=downscaled_data.sel(number=4)) ):.4f}")
print(f"MAE6: {np.nanmean(ps.crps_ensemble(observations=fine_data_test, forecasts=downscaled_data.sel(number=5)) ):.4f}")
print(f"MAE : {np.nanmean(ps.crps_ensemble(observations=fine_data_test, forecasts=downscaled_data.mean(dim='number')) ):.4f}")

In [ ]:
# np.corrcoef(fine_data_test.values.flatten(), downscaled_data.values.flatten())[0,1]

In [ ]:
from sklearn.metrics import r2_score
def compute_r2_with_nans(y_true, y_pred):
    # Mask for non-NaN values
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
    
    # Filter out NaN values
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    
    # Compute R2 score
    r2 = r2_score(y_true_filtered, y_pred_filtered)
    return r2
r2 = compute_r2_with_nans(fine_data_test.values.flatten(), downscaled_data.mean(dim='number').values.flatten())
r2

In [ ]:
# figs, axx = plt.subplots(figsize=(8,6))
# axx.scatter(fine_data_test.values.flatten(), downscaled_data.values.flatten(), s=0.1, alpha=0.9, color='C0')
# axx.plot([0,1000], [0,1000], color='red', alpha=0.9, zorder=1, ls='--')
# axx.set_xlim(-8,500)
# axx.set_ylim(-8,500)
# axx.set_xlabel('Observation', size=15)
# axx.set_ylabel('Downscaled', size=15)
# axx.text(45,400,f'$R^2: {r2:.2f}$', color='white', size=12)
# axx.grid(True, alpha=0.2, color='C1')

In [ ]:
crps_data = ps.crps_ensemble(observations=fine_data_test.sel(time=slice(date1, date2)), 
                             forecasts=downscaled_data.sel(time=slice(date1, date2)))
# Convert to xarray DataArray if necessary
crps_data = xr.DataArray(crps_data, dims=["time", "lat", "lon"], coords={"time": fine_data_test.sel(time=slice(date1, date2)).time,
                                                                         "lat": fine_data_test.lat, 
                                                                         "lon": fine_data_test.lon,
                                                                        })

# Define the colormap colors
colors_blues2black = [(0, 0, 0), (0, 0.90, 1)]  # Blue to Black
colors_RdBlBu = [(1, 0, 0), (0, 0, 0), (0, 0.90, 1)]  # Blue to Black

# Create the colormap
cmap_name = 'BluesToBlack'
blues_to_black = mcolors.LinearSegmentedColormap.from_list(cmap_name, colors_blues2black)
Rd_bl_Bu = mcolors.LinearSegmentedColormap.from_list('RdBlBu', colors_RdBlBu)


fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12,10), sharex=True, sharey=True,
                        subplot_kw=dict(projection=ccrs.PlateCarree(), facecolor='black'),
                        gridspec_kw={'wspace': -0.05, 'hspace': -0.45})

# Define the plots list
plots = [coarse_data_test, fine_data_test, downscaled_data.mean(dim='number'), crps_data]
cmap = [blues_to_black, blues_to_black, blues_to_black, 'afmhot']
levels = [np.arange(0,6.125,0.125), np.arange(0,6.125,0.125), np.arange(0,6.125,0.125), np.arange(0,6.125,0.125)]
title = ['a) Coarse','b) Fine','c) Downscaled','d) CRPS']
ticks = [np.arange(0,7,1), np.arange(0,7,1), np.arange(0,7,1), np.arange(0,7,1)]

for i, ax in enumerate(axs.flatten()):
    if i != (len(plots) - 1):
        plots[i].sel(time=slice(date1, date2)).mean('time').plot(cmap=cmap[i], levels=levels[i], 
                                                                 ax=ax, cbar_kwargs={'orientation':'vertical',
                                                                                    'pad':0.01, 'label':'',
                                                                                    'shrink':0.48, 'drawedges':False,
                                                                                    'ticks': ticks[i], },
                                                                 alpha=0.90)
    else:
        plots[-1].sel(time=slice(date1, date2)).mean('time').plot(cmap=cmap[i], levels=levels[i], 
                       ax=ax, cbar_kwargs={'orientation':'vertical',
                                           'pad':0.01, 'label':'',
                                           'shrink':0.48, 'drawedges':False,
                                           'ticks': ticks[i], },
                       alpha=0.90)
    
    ax.text(0, 55.2, f'{title[i]}', size=13, color='white')
    ax.coastlines(linewidth=1.5, color='white')
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=1.5, edgecolor='white')
    ax.patch.set_facecolor('black')

plt.show()